In [23]:
import pandas as pd
import os
import os.path as osp
import math
import numpy as np

In [2]:
train_df = pd.read_csv('./train_data.tsv',sep='\t')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20606 entries, 0 to 20605
Data columns (total 5 columns):
일자      20606 non-null int64
time    20606 non-null object
식사내용    20606 non-null object
매출일자    20606 non-null float64
수량      20606 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 805.0+ KB


In [3]:
train_df.head(4)

,일자,time,식사내용,매출일자,수량
0,20030301,아침,"과일샐러드,닭죽,돈육마늘장조림,떡만두국,부추김무침,쌀밥,딸기잼(중),비엔나구이,스크...",20030301.0,37.472924
1,20030301,저녁,"감자으깸샐러드,비프까스,스위트피클,쌀밥,옥수수스프,",20030301.0,19.566787
2,20030301,점심,"골뱅이야채무침,새우맛살튀김,쌀밥(사무직),열무겉절이,칼국수,",20030301.0,31.191336
3,20030302,아침,"계란죽,곤약멸치조림,김치국,마카로니샐러드,쌀밥,오징어회무침,딸기잼(중),삶은계란,야...",20030302.0,36.101083


In [4]:
doc_list = train_df['식사내용'].apply(lambda x: ' '.join(x[:-1].split(','))).tolist()

In [5]:
doc_list[0]

'과일샐러드 닭죽 돈육마늘장조림 떡만두국 부추김무침 쌀밥 딸기잼(중) 비엔나구이 스크램블에그(경양식) 야채샐러드 크림스프(경양식) 토스트&모닝빵'

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=2000,ngram_range=range(1,3))

In [11]:
X_food = vectorizer.fit_transform(doc_list)
print(X_food.shape)

(20606, 2000)


In [12]:
Y = train_df['수량'].values.reshape(-1,1)

In [13]:
print(X_food.shape, Y.shape)

(20606, 2000) (20606, 1)


In [15]:
vectorizer.transform(['과일샐러드 닭죽 돈육마늘장조림 떡만두국 부추김무침 쌀밥 딸기잼(중) 비엔나구이 스크램블에그(경양식) 야채샐러드 크림스프(경양식) 토스트&모닝빵'])

<1x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 23 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [17]:
model = LinearRegression()

In [18]:
cross_val_score(model,X=X_food,y=Y,cv=5, verbose=True,scoring='mean_squared_error')

/Users/myZZUNG/anaconda/envs/venvSKTB/lib/python3.5/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/myZZUNG/anaconda/envs/venvSKTB/lib/python3.5/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/myZZUNG/anaconda/envs/venvSKTB/lib/python3.5/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/myZZUNG/anaconda/envs/venvSKTB/lib/python3.5/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 

array([-212.03773667,  -85.62793624, -101.87575639,  -90.15484118,
        -89.72840309])

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_food, Y, test_size=0.2)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.4605470465683839

In [36]:
def do_RMSE(y_pred, y_gt):
    return math.sqrt(np.mean((y_pred - y_gt)**2))

In [37]:
print("RMSE: {0:.2f}".format(do_RMSE(y_pred=model.predict(X_test), y_gt=y_test)))

RMSE: 8.81


In [38]:
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
vectorizer = CountVectorizer(max_features=2000,ngram_range=range(1,3))

In [40]:
X_food = vectorizer.fit_transform(doc_list)
print(X_food.shape)

(20606, 2000)


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_food, Y, test_size=0.2)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.47483102517266557

In [42]:
print("RMSE: {0:.2f}".format(do_RMSE(y_pred=model.predict(X_test), y_gt=y_test)))

RMSE: 9.18
